In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
IMAGE_SIZE = [224, 224]

In [3]:
# Conneting to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Give dataset path
train_path = '/content/drive/MyDrive/Underwater/dataset/Train'
validation_path = '/content/drive/MyDrive/Underwater/dataset/Validation'

In [5]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
# Input shape
vgg.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>

In [7]:
# Freezing
for layer in vgg.layers:
  layer.trainable = False

In [8]:
# Number of classes
folders = glob('/content/drive/MyDrive/Underwater/dataset/Train/*')
print(len(folders))

3


In [9]:
# Creating a custom neural network model using a pre-trained VGG16 base

# Flatten the output from the VGG16 model to a 1D array
x = Flatten()(vgg.output)
# Add a Dense layer for classification with softmax activation
# The number of units in the Dense layer is set to the number of classes (folders)
prediction = Dense(len(folders), activation='softmax')(x)
# Create the final model by specifying input and output layers
model = Model(inputs=vgg.input, outputs=prediction)
# Display a summary of the model architecture
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
from keras import optimizers


adam = optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [11]:
# Training Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


In [12]:
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [13]:
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2594 images belonging to 3 classes.


In [14]:
validation_set = validation_datagen.flow_from_directory(validation_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 583 images belonging to 3 classes.


In [15]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint



checkpoint = ModelCheckpoint(filepath='VGG16_Training.h5', monitor='accuracy', verbose=2, save_best_only=True, mode='max')
callbacks = [checkpoint]

start = datetime.now()

model_history=model.fit_generator(
  train_set,
  validation_data=validation_set,
  epochs=15,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=2)


duration = datetime.now() - start
print("Training completed in time: ", duration)

<ipython-input-15-6c1cf058dad4>:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_history=model.fit_generator(


Epoch 1/15



Epoch 1: accuracy improved from -inf to 0.55000, saving model to VGG16_Training.h5
5/5 - 202s - loss: 8.5353 - accuracy: 0.5500 - val_loss: 9.3765 - val_accuracy: 0.7136 - 202s/epoch - 40s/step
Epoch 2/15


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: accuracy improved from 0.55000 to 0.77500, saving model to VGG16_Training.h5
5/5 - 27s - loss: 7.2901 - accuracy: 0.7750 - 27s/epoch - 5s/step
Epoch 3/15

Epoch 3: accuracy improved from 0.77500 to 0.89375, saving model to VGG16_Training.h5
5/5 - 25s - loss: 2.4143 - accuracy: 0.8938 - 25s/epoch - 5s/step
Epoch 4/15

Epoch 4: accuracy improved from 0.89375 to 0.90000, saving model to VGG16_Training.h5
5/5 - 23s - loss: 3.6104 - accuracy: 0.9000 - 23s/epoch - 5s/step
Epoch 5/15

Epoch 5: accuracy improved from 0.90000 to 0.92500, saving model to VGG16_Training.h5
5/5 - 23s - loss: 2.0880 - accuracy: 0.9250 - 23s/epoch - 5s/step
Epoch 6/15

Epoch 6: accuracy did not improve from 0.92500
5/5 - 23s - loss: 1.9098 - accuracy: 0.9187 - 23s/epoch - 5s/step
Epoch 7/15

Epoch 7: accuracy did not improve from 0.92500
5/5 - 19s - loss: 2.2204 - accuracy: 0.8813 - 19s/epoch - 4s/step
Epoch 8/15

Epoch 8: accuracy improved from 0.92500 to 0.93750, saving model to VGG16_Training.h5
5/5 - 1